# Shells Spawned by Web Servers
Web servers that spawn shell processes could be the result of a successfully placed web shell or an other attack

## Rule Content
```
- title: Shells Spawned by Web Servers
  id: 8202070f-edeb-4d31-a010-a26c72ac5600
  status: experimental
  description: Web servers that spawn shell processes could be the result of a successfully
    placed web shell or an other attack
  author: Thomas Patzke
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage:
      - '*\w3wp.exe'
      - '*\httpd.exe'
      - '*\nginx.exe'
      - '*\php-cgi.exe'
      Image:
      - '*\cmd.exe'
      - '*\sh.exe'
      - '*\bash.exe'
      - '*\powershell.exe'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  tags:
  - attack.privilege_escalation
  - attack.persistence
  - attack.t1100
  falsepositives:
  - Particular web applications may spawn a shell process legitimately
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:(*\\w3wp.exe OR *\\httpd.exe OR *\\nginx.exe OR *\\php\-cgi.exe) AND process_path.keyword:(*\\cmd.exe OR *\\sh.exe OR *\\bash.exe OR *\\powershell.exe))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()